In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
#import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [2]:
from funcoes import extract_stats

# Tentativa de com base equilibrada p%

In [3]:
p = 30

In [4]:
combinacoes_equilibradas = pd.read_csv("../combinacoes"+str(p)+".txt", header = 0, sep=",")

In [5]:
X = combinacoes_equilibradas[["mse", "iss", "mse_centro", "iss_centro",
            "mse_canny", "iss_canny", "mse_canny_centro", "iss_canny_centro",
            "mse_skeleton", "iss_skeleton", "mse_skeleton_centro", "iss_skeleton_centro",
            "imgA_mean", "imgB_mean", "imgA_var", "imgB_var",
            #imgA_contraste, imgB_contraste,
            #imgA_angular_momentum, imgB_angular_momentum,
            "imgA_entropy", "imgB_entropy"]]

y = combinacoes_equilibradas["resposta"]

In [6]:
#from sklearn.manifold import Isomap
#iso = Isomap(n_components = 2)
#data_projected = iso.fit_transform(X)

In [7]:
#data_projected.shape

In [8]:
#plt.scatter(data_projected[:, 0], data_projected[:, 1], c=y,
#            edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral'))
#plt.colorbar(label='digita label', ticks=range(10))
#plt.clim(-0.5, 9.5)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## SVM

In [10]:
%%time
#from sklearn.svm import SVC
#clf = SVC(max_iter = 100000) # kernel = 'poly', degree = 2, 

#SVMparams = { 'kernel' : ('linear', 'poly', 'rbf')} #, 'sigmoid')}

#svr = SVC()
#clf = GridSearchCV(svr, SVMparams)

#clf.fit(X_train, y_train)
#print( "Score: {0}".format(clf.score(X_test, y_test)) )
#print( "Best parameters: " + clf.best_params_ )

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs


## Decision Tree

In [11]:
%%time
from sklearn import tree

DTparams = { 'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 3, 7, 40)}

svr = tree.DecisionTreeClassifier()
clf1 = GridSearchCV(svr, DTparams) #criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clf1.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clf1.score(X_test, y_test)) )
print( "Best parameters: " + str(clf1.best_params_) )

Score: 0.7553846153846154
Best parameters: {'criterion': 'entropy', 'min_samples_leaf': 7, 'min_samples_split': 40}
CPU times: user 56 s, sys: 328 ms, total: 56.4 s
Wall time: 56.8 s


# Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [13]:
%%time

RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.8020512820512821
Best parameters: {'n_estimators': 10, 'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 10}
CPU times: user 3min 54s, sys: 2.94 s, total: 3min 57s
Wall time: 3min 58s


In [29]:
%%time

from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier(criterion = 'entropy', n_estimators=10, min_samples_leaf=5,
                                       min_samples_split=10)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )

Score: 0.7592592592592593
CPU times: user 1.22 s, sys: 8 ms, total: 1.23 s
Wall time: 1.23 s


## Logistic Regression

In [15]:
%%time
from sklearn.linear_model import LogisticRegression

LRparams = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag')}

svr = LogisticRegression()
clfLR = GridSearchCV(svr, LRparams)
clfLR.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfLR.score(X_test, y_test)) )
print( "Best parameters: " + str(clfLR.best_params_) )

/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The li

Score: 0.7505982905982906
Best parameters: {'solver': 'newton-cg'}
CPU times: user 34.5 s, sys: 58.3 s, total: 1min 32s
Wall time: 33.9 s


/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


# Multi-layer Perceptron classifier

In [16]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
            'activation' : ('identity', 'logistic', 'tanh', 'relu'),
            'solver' : ('lbfgs', 'sgd', 'adam')}

svr = MLPClassifier()
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.29555555555555557
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 2.02 s, sys: 4.51 s, total: 6.53 s
Wall time: 2.62 s


# Tentativa com base de p = 50%

In [17]:
p = 50

In [18]:
combinacoes_equilibradas = pd.read_csv("../combinacoes"+str(p)+".txt", header = 0, sep=",")

In [19]:
X = combinacoes_equilibradas[["mse", "iss", "mse_centro", "iss_centro",
            "mse_canny", "iss_canny", "mse_canny_centro", "iss_canny_centro",
            "mse_skeleton", "iss_skeleton", "mse_skeleton_centro", "iss_skeleton_centro",
            "imgA_mean", "imgB_mean", "imgA_var", "imgB_var",
            #imgA_contraste, imgB_contraste,
            #imgA_angular_momentum, imgB_angular_momentum,
            "imgA_entropy", "imgB_entropy"]]

y = combinacoes_equilibradas["resposta"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## SVM

In [21]:
%%time
#from sklearn.svm import SVC
#clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

#SVMparams = { 'kernel' : ('linear', 'poly', 'rbf')} #, 'sigmoid')}

#svr = SVC()
#clf = GridSearchCV(svr, SVMparams)

#clf.fit(X_train, y_train)
#print( "Score: {0}".format(clf.score(X_test, y_test)) )
#print( "Best parameters: " + clf.best_params_ )

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.2 µs


## Decision Tree

In [22]:
%%time
from sklearn import tree

DTparams = { 'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 3, 7, 40)}

svr = tree.DecisionTreeClassifier()
clf1 = GridSearchCV(svr, DTparams) #criterion = "gini", min_samples_split = 80, min_samples_leaf = 3

clf1.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clf1.score(X_test, y_test)) )
print( "Best parameters: " + str(clf1.best_params_) )

Score: 0.7173789173789173
Best parameters: {'criterion': 'gini', 'min_samples_leaf': 3, 'min_samples_split': 40}
CPU times: user 28.3 s, sys: 244 ms, total: 28.5 s
Wall time: 28.6 s


# Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [24]:
%%time

RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.7683760683760684
Best parameters: {'n_estimators': 10, 'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 10}
CPU times: user 2min, sys: 2.32 s, total: 2min 2s
Wall time: 2min 3s


## Logistic Regression

In [26]:
%%time
from sklearn.linear_model import LogisticRegression

LRparams = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag')}

svr = LogisticRegression()
clfLR = GridSearchCV(svr, LRparams)
clfLR.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfLR.score(X_test, y_test)) )
print( "Best parameters: " + str(clfLR.best_params_) )

/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ricardo/envs/facerecog/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The li

Score: 0.6373219373219373
Best parameters: {'solver': 'newton-cg'}
CPU times: user 16.8 s, sys: 25.5 s, total: 42.3 s
Wall time: 11.9 s


/home/ricardo/envs/facerecog/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


# Multi-layer Perceptron classifier

In [27]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
            'activation' : ('identity', 'logistic', 'tanh', 'relu'),
            'solver' : ('lbfgs', 'sgd', 'adam')}

svr = MLPClassifier()
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.5116809116809117
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 1.38 s, sys: 2.24 s, total: 3.62 s
Wall time: 1.26 s


# Binarizando o melhor modelo

In [ ]:
#imgA = skio.imread("../letras/caracter1.png")
#imgB = skio.imread("../letras/caracter15.png")

#mse, iss, mse_centro, iss_centro,\
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,\
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro = extract_stats(imgA, imgB)
        
#print(clfRF_balanced.predict( [[mse, iss, mse_centro, iss_centro,
#			mse_canny, iss_canny, mse_canny_centro, iss_canny_centro,
#			mse_skeleton, iss_skeleton, mse_skeleton_centro, iss_skeleton_centro]]))

In [30]:
joblib.dump(clfRF_balanced, 'classifier_balanced.pkl') 

['classifier_balanced.pkl']